In [2]:
import numpy as np
import os
import orjson
import collections

from transformers import AutoTokenizer

In [12]:
load_dir = '/projects/abeb4417/multilingual_analysis/extract_hr_nn/outputs/efficient_test/che/'

loaded_data = {}

for found_file in os.listdir(load_dir):

    full_file_path = os.path.join(load_dir, found_file)

    if '.npy' in found_file:

        v_name = found_file[:-4]
        v = np.load(full_file_path)

        loaded_data[v_name] = v
    
    elif '.jsonl' in found_file:
        v_name = found_file[:-6]
        with open(full_file_path, 'r') as f:
            for line in f:
                v = orjson.loads(line.strip())
                break #  Assume only 1 line of information, must edit if this changes
        loaded_data[v_name] = v


tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')


loading configuration file config.json from cache at /home/abeb4417/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/e73636d4f797dec63c3081bb6ed5c7b0bb3f2089/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.30.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading file sentencepiece.bpe.model from cache at /home/abeb4417/.cache/huggingface/hub/models--xlm-roberta-ba

In [13]:
""" 
source_hidden_reps: (32753, 768)
source_indices: (31253,)
source_input_ids: (32753,)
source_langs_input_id: (32753,)
source_langs_line: (31253,)


target_hidden_reps: (8834, 768) 
target_indices: (8334,) #Main indexer for these functions. Maps index --> line number
target_input_ids: (8834,) 
target_langs: (8334,)

distances: (8834, 10)
neighbors_indices: (8834, 10)


"""

keys = list(loaded_data.keys())
keys.sort()


for k in keys:
    v = loaded_data[k]
    if isinstance(v, np.ndarray):
        print(f'{k}: {v.shape}')


distances: (31745, 10)
neighbors_indices: (31745, 10)
source_hidden_reps: (51031, 768)
source_indices: (51031,)
source_input_ids: (51031,)
source_langs: (51031,)
target_hidden_reps: (31745, 768)
target_indices: (31745,)
target_input_ids: (31745,)
target_langs: (31745,)


In [19]:
# Helper Functions

def get_token_idx_from_line(line_idx, indices):
    return np.where(indices == line_idx)

def get_token_input_ids_from_token_ids(token_indices, input_ids):
    return input_ids[token_indices]

def get_token_input_ids_from_line(line_idx, indices, input_ids):
    return input_ids[np.where(indices == line_idx)]

def get_all_line_idx(indices):
    return np.unique(indices)

def get_first_available_line(indices):
    return indices[0]



In [21]:
line_idx = 26281

target_token_ids = get_token_idx_from_line(line_idx, loaded_data['target_indices'])[0] # The tokens in line 0 are mapped to the indices 0-9
print(target_token_ids)
# use these indices to look up their input ids:

target_input_ids = get_token_input_ids_from_token_ids(target_token_ids, loaded_data['target_input_ids'])
print(target_input_ids)

# Sanity check: convert these ids to the tokens:
print(tokenizer.convert_ids_to_tokens(target_input_ids))

# For each token, lets get their nearest neighbors

for i in range(target_token_ids.shape[0]):
    print('#####'*50)

    target_token_id = target_token_ids[i]
    target_token_input_id = target_input_ids[i]

    print(target_token_id)
    print(target_token_input_id)
    print(tokenizer.convert_ids_to_tokens([target_token_input_id]))

    neighbors = loaded_data['neighbors_indices'][target_token_id]
    print(neighbors)

    neighbor_input_ids = loaded_data['source_input_ids'][neighbors]
    print(neighbor_input_ids)
    print(tokenizer.convert_ids_to_tokens(neighbor_input_ids))

    print(loaded_data['source_langs'][neighbors])





[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74]
[  2739   7309  21678  34879  55461    212   1721    551      6   4019
  49326      6  78234    197  27258  17831   2893   2144    983   4753
    135    244   1041 135999  49971  25485   1041   1976    212      6
  58138   4737   7309     59   7736  55953  36175      6      5   3743
     59   1721    105  33122    135    244   1041  16962   1041    547
    551      6      4   2744  78234     59   1721  29376    804   6168
    244   7309     89   5004   2744   7309     59   1041    244  18654
  39577  82573    551      6      5]
['▁Х', 'І', 'ара', '▁ду', '▁шу', 'на', '▁Да', 'ла', '▁', 'лер', 'рина', '▁', 'къ', 'о', 'бал', 'вин', 'чу', '▁І', 'ий', 'сан', '▁с', 'х', 'ь', 'авала', 'рх', '▁ла', 'ь', 'ц', 'на', '▁', 'долу', '▁д', 'І', 'а', 'яз', 'дина', 'рг', '▁', '

In [15]:
all_line_indices = np.unique(loaded_data['target_indices'])

lang_distribution = collections.Counter()

for line_idx in all_line_indices:

    target_token_ids = get_token_idx_from_line(line_idx, loaded_data['target_indices'])[0] # The tokens in line 0 are mapped to the indices 0-9
    # print(target_token_ids)
    # use these indices to look up their input ids:

    target_input_ids = get_token_input_ids_from_token_ids(target_token_ids, loaded_data['target_input_ids'])
    # print(target_input_ids)

    # Sanity check: convert these ids to the tokens:
    # print(tokenizer.convert_ids_to_tokens(target_input_ids))

    # For each token, lets get their nearest neighbors

    for i in range(target_token_ids.shape[0]):
        # print('#####'*50)

        target_token_id = target_token_ids[i]
        target_token_input_id = target_input_ids[i]

        # print(target_token_id)
        # print(target_token_input_id)
        # print(tokenizer.convert_ids_to_tokens([target_token_input_id]))

        neighbors = loaded_data['neighbors_indices'][target_token_id]
        # print(neighbors)

        neighbor_input_ids = loaded_data['source_input_ids'][neighbors]
        # print(neighbor_input_ids)
        # print(tokenizer.convert_ids_to_tokens(neighbor_input_ids))

        # print(loaded_data['source_langs_input_id'][neighbors])
        lang_distribution.update(loaded_data['source_langs'][neighbors])


print(lang_distribution)


Counter({'rus': 264388, 'eng': 43471, 'ess': 9591})
